In [46]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import itertools
import pickle, gzip
import gc

In [59]:
train_series = pd.read_csv('Data/training_sample_set.csv')
train_metadata = pd.read_csv('Data/training_sample_set_metadata.csv')

simple_features = train_series.groupby(
    ['object_id', 'passband'])['flux', 'flux_err'].agg(
    ['mean', 'median', 'max', 'min', 'std']).unstack('passband')


#construct time series using binned observations:
ts_mod = train_series[['object_id', 'mjd', 'passband', 'flux', 'flux_err']].copy()
#bin by 5 days, reducing the size of data but still giving a time series
ts_mod['mjd_d5'] = (ts_mod['mjd'] / 5).astype(int)
ts_mod = ts_mod.groupby(['object_id', 'mjd_d5', 'passband'])['flux', 'flux_err'].mean().reset_index()

#pivotting
ts_piv = pd.pivot_table(ts_mod, 
                        index='object_id', 
                        columns=['mjd_d5', 'passband'], 
                        values=['flux','flux_err'],
                        dropna=False)

gc.enable()
#Read in metadata
metadata_train=pd.read_csv('Data/training_sample_set_metadata.csv')
#Not using gal_l and gal_b features
del metadata_train['gal_l'], metadata_train['gal_b']
#Bin into ddf and non-ddf training
#ddf = metadata_train[(metadata_train['ddf'] == 1)]
#y_train_ddf = ddf['target']
#non_ddf = metadata_train[(metadata_train['ddf'] == 0)]
#y_train_non_ddf = non_ddf3['target']
#del metadata_train, ddf['target'], non_ddf['target'], ddf['ddf'], non_ddf['ddf']
gc.collect()
#ddf.head(10)



105